In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from scipy import sparse
from sklearn.cross_validation import train_test_split

from operator import itemgetter
import os
import random
import subprocess
import pickle
import re
from time import time

from numpy.linalg import svd

/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import logging
reload(logging)
logging.basicConfig(format="%(asctime)s   %(message)s", level=logging.INFO, datefmt="%H:%M:%S")

### Просмотр данных

In [3]:
word_type = {}
concepts = []
entities = []
with open("SemEval18-Task9/training/data/1A.english.training.data.txt") as trainF:
    for line in trainF.readlines():
        infa = line.strip().split('\t')
        word = infa[0].lower()
        type_word = infa[1]
        
        if type_word == 'Concept':
            concepts.append(word)
        else:
            entities.append(word)

In [4]:
print "<< CONCEPTS >>"
print 


conc = 0
for w in concepts:
    conc += 1
    if conc <= 20:
        print w
    
        
print "..."
print 
print "All: " + str(conc)

<< CONCEPTS >>

blackfly
abhorrence
tropical storm
militarization
pollution
photomicrograph
swamp gum
song
wing
cumulus
silver
sand verbena
louse
sodium nitrite
navigator
gasworks
farming
failure rate
chess
floorshow
...

All: 979


In [5]:
print "<< ENTITIES >>"
print 


ent = 0
for w in entities:
    ent += 1
    if ent <= 20:
        print w
    
        
print "..."
print 
print "All: " + str(ent)

<< ENTITIES >>

turonian
hero
carly fiorina
erna brodber
raoul dufy
skara brae
murder one
emmett tyrrell
kim il-sung university
sachs harbour
myrdal
crowds and power
a1 motorway
muhammad ali jinnah
bennington
helen of troy
north carolina
forest way
younger dryas
new zealand
...

All: 521


In [6]:
hipon_type = []
with open("SemEval18-Task9/training/data/1A.english.training.data.txt") as trainF:
    for line in trainF.readlines():
        infa = line.strip().split('\t')
        word = infa[0].lower()
        type_word = infa[1]
        hipon_type.append([word, type_word])
hipon_type = np.array(hipon_type)

In [7]:
hipons = np.array(hipon_type)[:,0]

In [8]:
hipon_test_type = []
with open("SemEval18-Task9/trial/data/1A.english.trial.data.txt") as trainF:
    for line in trainF.readlines():
        infa = line.strip().split('\t')
        word = infa[0].lower()
        type_word = infa[1]
        hipon_test_type.append([word, type_word])
hipon_test_type = np.array(hipon_test_type)

hipons_test = np.array(hipon_test_type)[:,0]

In [9]:
hipers = []
with open("SemEval18-Task9/training/gold/1A.english.training.gold.txt") as trainF:
    for line in trainF.readlines():
        words = map(lambda w: w.lower(), line.strip().split('\t'))
        hipers.append(words)

In [10]:
print "<< Примеры гипоним: гиперонимы >>"
print

i = 0 
for O, E in zip(hipon_type[:,0], hipers):
    line = O + ': '
    for a in E[:-1]:
        line += a + ', '
    line += E[-1]
    print line
    print
    i += 1
    if i > 10:
        break
        
print "..."
print 
print "All: " + str(len(hipers))

<< Примеры гипоним: гиперонимы >>

blackfly: homopterous insect, insect

turonian: technical specification, geologic timescale, physical property, geological period, magnitude, unit of time, geological time, geologic time

abhorrence: distaste, hatred, hate, disgust

tropical storm: atmosphere, windstorm, violent storm, air current, atmospheric state, density, current of air, storm damage, atmospheric phenomenon, storm, cyclone, natural phenomenon, tempest, wind

militarization: social control

pollution: dirtiness, dirtying, environmental condition, impurity, sanitary condition, uncleanness

photomicrograph: picture, photograph, photo, image, pic, digital image

swamp gum: eucalyptus, plant, eucalyptus tree, angiosperm, gum, gum tree, woody plant, eucalypt

song: possession, sound, work of art, musical composition, vocal music, musical work, piece of music, human language, speech, signaling

wing: emblem, technical specification, form, specifications, symbolization, aerofoil, air unit

In [11]:
vocab_nE = {}
nE_vocab = {}
with open("SemEval18-Task9/vocabulary/1A.english.vocabulary.txt") as vocF:
    for i, line in enumerate(vocF.readlines()):
        word = line.strip().lower()
        vocab_nE[word] = i
        nE_vocab[i] = word

In [12]:
print "Кол-во слов(фраз) в словаре: " + str(len(vocab_nE))

Кол-во слов(фраз) в словаре: 218755


In [13]:
def predict(train, pred):
    return subprocess.check_output("python ./SemEval18-Task9/task9-scorer.py " + train + " " + pred)

### Patterns Hearest

In [12]:
# From https://github.com/mmichelsonIF/hearst_patterns_python/blob/master/

import re
import string
import nltk
from nltk.tag.perceptron import PerceptronTagger

class HearstPatterns(object):

    def __init__(self, extended = False):
        self.__chunk_patterns = r""" #  helps us find noun phrase chunks
                NP: {<DT|PP\$>?<JJ>*<NN>+}
                    {<NNP>+}
                    {<NNS>+}
        """

        self.__np_chunker = nltk.RegexpParser(self.__chunk_patterns) # create a chunk parser 

        # now define the Hearst patterns
        # format is <hearst-pattern>, <general-term>
        # so, what this means is that if you apply the first pattern, the firsr Noun Phrase (NP)
        # is the general one, and the rest are specific NPs
        self.__hearst_patterns = [
                ("(NP_\w+ (, )?such as (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(such NP_\w+ (, )?as (NP_\w+ ?(, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?other NP_\w+)", "last"),
                ("(NP_\w+ (, )?including (NP_\w+ ?(, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?especially (NP_\w+ ?(, )?(and |or )?)+)", "first"),
            ]

        if extended:
            self.__hearst_patterns.extend([
                ("((NP_\w+ ?(, )?)+(and |or )?any other NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?some other NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?is a NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?was a NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?were a NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?are a NP_\w+)", "last"),
                ("(NP_\w+ (, )?like (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("such (NP_\w+ (, )?as (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?like other NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?one of the NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?one of these NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?one of those NP_\w+)", "last"),
                ("examples of (NP_\w+ (, )?is (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("examples of (NP_\w+ (, )?are (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?are examples of NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?is example of NP_\w+)", "last"),
                ("(NP_\w+ (, )?for example (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?wich is called NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?which is named NP_\w+)", "last"),
                ("(NP_\w+ (, )?mainly (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?mostly (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?notably (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?particularly (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?principally (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?in particular (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?except (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?other than (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?e.g. (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?i.e. (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?a kind of NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?kinds of NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?form of NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?forms of NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?which looks like NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?which sounds like NP_\w+)", "last"),
                ("(NP_\w+ (, )?which are similar to (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?which is similar to (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?examples of this is (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?examples of this are (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?types (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )? NP_\w+ types)", "last"),
                ("(NP_\w+ (, )?whether (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(compare (NP_\w+ ?(, )?)+(and |or )?with NP_\w+)", "last"),
                ("(NP_\w+ (, )?compared to (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?among them (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?as NP_\w+)", "last"),
                ("(NP_\w+ (, )? (NP_\w+ ? (, )?(and |or )?)+ for instance)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?sort of NP_\w+)", "last"),
            ])

        self.__pos_tagger = PerceptronTagger()
        
    def prepare(self, rawtext):
        try:
            sentences = nltk.sent_tokenize(rawtext.strip()) # NLTK default sentence segmenter
            sentences = [nltk.word_tokenize(sent) for sent in sentences] # NLTK word tokenizer
            sentences = [self.__pos_tagger.tag(sent) for sent in sentences] # NLTK POS tagger
        except:
            sentences = []
        return sentences

    def chunk(self, rawtext):
        sentences = self.prepare(rawtext.strip())

        all_chunks = []
        for sentence in sentences:
            chunks = self.__np_chunker.parse(sentence) # parse the example sentence
            #for chunk in chunks:
            #   print(str(chunk))
            all_chunks.append(self.prepare_chunks(chunks))
        return all_chunks

    def prepare_chunks(self, chunks):
        # basically, if the chunk is NP, keep it as a string taht starts w/ NP and replace " " with _
        # otherwise, keep the word.
        # remove punct
        # this is all done to make it super easy to apply the Hearst patterns...

        terms = []
        for chunk in chunks:
            label = None
            try: # gross hack to see if the chunk is simply a word or a NP, as we want. But non-NP fail on this method call
                label = chunk.label()
            except:
                pass

            if label is None: #means one word...
                token = chunk[0]
                pos = chunk[1]
                if pos in ['.', ':', '-', '_']:
                    continue
                terms.append(token)
            else:
                np = "NP_"+"_".join([a[0] for a in chunk]) #This makes it easy to apply the Hearst patterns later
                terms.append(np)
        return ' '.join(terms)

    """
        This is the main entry point for this code.
        It takes as input the rawtext to process and returns a list of tuples (specific-term, general-term)
        where each tuple represents a hypernym pair.

    """
    def find_hyponyms(self, rawtext):

        hyponyms = []
        np_tagged_sentences = self.chunk(rawtext)

        for raw_sentence in np_tagged_sentences:
            # two or more NPs next to each other should be merged into a single NP, it's a chunk error

            # find any N consecutive NP_ and merge them into one...
            # So, something like: "NP_foo NP_bar blah blah" becomes "NP_foo_bar blah blah"
            sentence = re.sub(r"(NP_\w+ NP_\w+)+", lambda m: m.expand(r'\1').replace(" NP_", "_"), raw_sentence)

            for (hearst_pattern, parser) in self.__hearst_patterns:
                matches = re.search(hearst_pattern, sentence)
                if matches:
                    match_str = matches.group(0)

                    nps = [a for a in match_str.split() if a.startswith("NP_")]

                    if parser == "first":
                        general = nps[0]
                        specifics = nps[1:]
                    else:
                        general = nps[-1]
                        specifics = nps[:-1]
                        #print(str(general))
                        #print(str(nps))

                    for i in range(len(specifics)):
                        #print("%s, %s" % (specifics[i], general))
                        hyponyms.append((self.clean_hyponym_term(specifics[i]), self.clean_hyponym_term(general)))

        return hyponyms


    def clean_hyponym_term(self, term):
        # good point to do the stemming or lemmatization
        return term.replace("NP_","").replace("_", " ")

hearst_patterns = HearstPatterns()
hearst_patterns.find_hyponyms("There are works by such authors as Herrick, Goldsmith, and Shakespeare.")

In [14]:
def group_by_O(ac):
    groups = {}
    for v, O, E in zip(ac.data, ac.row, ac.col):
        groups.setdefault(O, [])
        groups[O].append([E, v])
    return groups

In [ ]:
# O_array = []
# E_array = []
# Value_array = []

# n_line = 0
# cnt_found_pairs = 0
# cnt_get_pairs = 0

# vocab_nO = {}
# nO_vocab = {}
# with open('UMBC_tokenized.txt') as f_text:
        
#     while True:
#         n_line += 1
        
# #         if cnt_get_pairs > 10:
# #             break
        
#         line = f_text.readline()
#         if len(line) == 0:
#             break
            
#         O_E = hearst_patterns.find_hyponyms(line)
#         if len(O_E) != 0:
#             for (O, E) in O_E:
#                 E = E.lower()
#                 O = O.lower()
#                 cnt_found_pairs += 1
#                 if E in vocab_nE:
#                     E_array.append(vocab_nE[E])
                    
#                     if O not in vocab_nO:
#                         vocab_nO[O] = len(vocab_nO)
#                         nO_vocab[len(vocab_nO) - 1] = O
#                     O_array.append(vocab_nO[O])
                    
#                     Value_array.append(1)
#                     cnt_get_pairs += 1
        
#         if n_line % 30000 == 0:
#             logging.info("line: %d, found_pairs %d, get_pairs %d" %(n_line, cnt_found_pairs, cnt_get_pairs))
            
#             with open('dumpEO', 'w') as dumpF:
#                 matrix = sparse.coo_matrix((Value_array, (O_array, E_array))).tocsr()
#                 dumpF.write(pickle.dumps((matrix, vocab_nO, nO_vocab)))
# #                 dumpF.write(pickle.dumps(nO_vocab) + "\n")
# #                 dumpF.write(pickle.dumps(vocab_nO))
        

21:49:27   line: 30000, found_pairs 1145, get_pairs 271
21:50:44   line: 60000, found_pairs 1735, get_pairs 397
21:52:08   line: 90000, found_pairs 1973, get_pairs 448
21:53:31   line: 120000, found_pairs 2256, get_pairs 525
21:54:56   line: 150000, found_pairs 2671, get_pairs 638
21:56:16   line: 180000, found_pairs 3399, get_pairs 793
21:57:32   line: 210000, found_pairs 4343, get_pairs 979
21:58:47   line: 240000, found_pairs 5301, get_pairs 1201
22:00:07   line: 270000, found_pairs 6570, get_pairs 1414
22:01:20   line: 300000, found_pairs 7293, get_pairs 1522
22:02:35   line: 330000, found_pairs 8048, get_pairs 1687
22:03:55   line: 360000, found_pairs 9281, get_pairs 1929
22:05:19   line: 390000, found_pairs 10481, get_pairs 2195
22:06:40   line: 420000, found_pairs 11534, get_pairs 2368
22:08:06   line: 450000, found_pairs 12532, get_pairs 2562
22:09:11   line: 480000, found_pairs 12867, get_pairs 2638
22:10:23   line: 510000, found_pairs 13316, get_pairs 2753
22:11:30   line: 54

In [25]:
# coo_Matrix = sparse.coo_matrix((Value_array, (O_array, E_array))).tocsr()
# ac = coo_Matrix.tocoo()

In [27]:
# for v, O, E in zip(ac.data, ac.row, ac.col):
#     print nO_vocab[O] + ', ', nE_vocab[E], v

successful completion,  newfoundland 1
information,  decision 1
retroactive effect,  progress 1


In [30]:
with open('dumpEO') as dumpF:
    coo_Matrix, vocab_nO, nO_vocab = pickle.loads(dumpF.read())
    ac = coo_Matrix.tocoo()

In [33]:
sums = []
with open("predict_Hearst", "w") as predF:
    groups = group_by_O(ac)

    for O_gold in hipon_type[:,0]:
        if O_gold in vocab_nO:
            num_O_gold = vocab_nO[O_gold]
            sums.append(np.array(groups[num_O_gold])[:,1].sum())
            Es = np.array(sorted(groups[num_O_gold], key=itemgetter(1), reverse=True))[:,0]
        else:
            sums.append(0)
            Es = np.array([])
        
        line = map(lambda x: nE_vocab[x], Es)
        
        predF.write("\t".join(line) + "\n")
        
print np.array(sums).mean()

0.434666666667


In [34]:
!python SemEval18-Task9/task9-scorer.py ./SemEval18-Task9/training/gold/1A.english.training.gold.txt ./predict_Hearst

MRR: 0.00505555555556
MAP: 0.0017171345938
P@1: 0.00333333333333
P@3: 0.00233333333333
P@5: 0.00178888888889
P@15: 0.00119837199837


In [ ]:

def merge_dumpEO():
    for file_name in os.listdir('./'):
        if 'dumpEO' in file_name:
            

### PMI + SVD + SVM

In [14]:
pat_word = re.compile('\w+')
def prepare(text):
        try:
            sentences = re.findall(pat_word, text)
        except:
            sentences = []
        
#         push = False
#         for word in sentences:
#             word = word.lower()
#             if word in vocab_nE or word in hipons:
#                 push = True
#                 break
                
#         if push:
#             return sentences
#         else:
#             return []

        return sentences

In [ ]:
Sent_array = []
Word_array = []
Value_array = []

w_s_v = {}
# common_array = []

n_line = 1
cnt_found_pairs = 0
cnt_get_pairs = 0

vocab_nW = {}
nW_vocab = {}

with open('UMBC_tokenized.txt') as f_text:
        
    while True:
        
#         if len(vocab_nW) > 1000:
#             break
        
        line = f_text.readline()
        if len(line) == 0:
            break
            
        sent = prepare(line)
        for word in sent:
            word = word.lower()
#             if word in vocab_nE or word in hipons:
#                 push = True
                
            Sent_array.append(n_line)

            if word not in vocab_nW:
                vocab_nW[word] = len(vocab_nW)
                nW_vocab[len(vocab_nW) - 1] = word
            Word_array.append(vocab_nW[word])

            Value_array.append(1)

#             key = str(n_line) + '_' + str(vocab_nW[word])
#             key = (n_line, vocab_nW[word])
#             w_s_v.setdefault(key, 0)
#             w_s_v[key] += 1

        if n_line % 500000 == 0:
            logging.info("line: %d, words %d" %(n_line, len(vocab_nW)))
            
        if n_line % 2000000 == 0:
            if len(Value_array) != 0:
                with open('dumpPMI_total_' + str(n_line/2000000), 'w') as dumpF:
                    matrix = sparse.coo_matrix((Value_array, (Word_array, Sent_array))).tocsr()
                    dumpF.write(pickle.dumps(matrix))

                with open('dumpWords_total_' + str(n_line/2000000), 'w') as dumpF:
                    dumpF.write(pickle.dumps((vocab_nW, nW_vocab)))

                Sent_array = []
                Word_array = []
                Value_array = []

                vocab_nW = {}
                nW_vocab = {}
            
        n_line += 1

10:22:20   line: 500000, words 0
10:22:37   line: 1000000, words 0
10:22:53   line: 1500000, words 0
10:23:09   line: 2000000, words 0
10:23:26   line: 2500000, words 0
10:23:42   line: 3000000, words 0
10:23:58   line: 3500000, words 0
10:24:14   line: 4000000, words 0
10:24:31   line: 4500000, words 0
10:24:47   line: 5000000, words 0
10:25:03   line: 5500000, words 0
10:25:19   line: 6000000, words 0
10:25:35   line: 6500000, words 0
10:25:51   line: 7000000, words 0
10:26:07   line: 7500000, words 0
10:26:22   line: 8000000, words 0
10:26:39   line: 8500000, words 0
10:26:55   line: 9000000, words 0
10:27:11   line: 9500000, words 0
10:27:29   line: 10000000, words 0
10:27:46   line: 10500000, words 0
10:28:02   line: 11000000, words 0
10:28:18   line: 11500000, words 0
10:28:34   line: 12000000, words 0
10:28:50   line: 12500000, words 0
10:29:07   line: 13000000, words 0
10:29:23   line: 13500000, words 0
10:29:40   line: 14000000, words 0
10:29:58   line: 14500000, words 0
10:30

In [ ]:
with open('dumpPMI', 'w') as dumpF:
    matrix = sparse.coo_matrix((Value_array, (Word_array, Sent_array))).tocsr()
    dumpF.write(pickle.dumps(matrix))
with open('dumpWords', 'w') as dumpF:
    dumpF.write(pickle.dumps((vocab_nW, nW_vocab)))

In [63]:
with open('dumpPMI') as dumpF:
    coo_Matrix = pickle.loads(dumpF.read())
    ac = coo_Matrix.tocoo()
    
with open('dumpWords') as dumpF:
    vocab_nW, nW_vocab = pickle.loads(dumpF.read())

In [25]:
Sent_array = []
Word_array = []
Value_array = []

for (w, s), v in w_s_v.items():
    Sent_array.append(s)
    Word_array.append(w)
    Value_array.append(v)

In [26]:
matrix = sparse.coo_matrix((Value_array, (Word_array, Sent_array))).tocsr()
# for v, word, sent in zip(ac.data, ac.row, ac.col):
#     print nW_vocab[word] + ', ', sent, v

In [ ]:
word_sent_ij = {}
word_i = {}
sent_j = {}

for v, word, sent in zip(ac.data, ac.row, ac.col):
        

### Word2Vec

In [15]:
import gensim
from lightgbm import LGBMRanker

18:32:04   'pattern' package found; tag filters are available for English


In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('./modelsW2V/GoogleNews-vectors-negative300.bin', binary=True)  

18:23:19   loading projection weights from ./modelsW2V/GoogleNews-vectors-negative300.bin


In [27]:
found = 0
allO = 0
for O, typeO in hipon_type:
    if typeO == 'Concept':
        allO += 1
        try:
            model.wv[O]
            found += 1
        except:
            pass
print found, allO
print float(found) / allO

661 979
0.67517875383


In [28]:
found = 0
allO = 0
for O, typeO in hipon_test_type:
    if typeO == 'Concept':
        allO += 1
        try:
            model.wv[O]
            found += 1
        except:
            pass
print found, allO
print float(found) / allO

15 30
0.5


In [202]:
def diff(v1, v2):
    return (v1 - v2).reshape(1, len(v1))[0]

In [204]:
diff(np.array([1,2,3]), np.array([4,5,6])).shape

(3,)

In [113]:
def create_DF_Bless():
#     for O, typeO, Es in zip(hipon_type, hipers):
#         if typeO == 'Concept':
#             try:
#                 vecO = model.wv[O]
#                 for E in Es:
#                     try:
#                         vecE = model.wv[E]
    X = []
    y = []
    cnt_rand, cnt_hyp = 0,0
    with open('./sim-eval_datasets/datasets/bless.csv') as blessF:
        for line in blessF.readlines():
            parts = line.strip().split(';')
            O = parts[0]
            E = parts[1]
            typeOE = parts[2]
            
            if typeOE == 'random' and cnt_hyp > cnt_rand:
                try:
                    O_vec = model.wv[O]
                    E_vec = model.wv[E]
                    X.append(diff(O_vec, E_vec))
                    y.append(0)
                    cnt_rand += 1
                except:
                    continue
                    
            if typeOE == 'hyper':
                try:
                    O_vec = model.wv[O]
                    E_vec = model.wv[E]
                    X.append(diff(O_vec, E_vec))
                    y.append(1)
                    cnt_hyp += 1
                except:
                    continue
                    
    return X, y

In [341]:
random.seed(37)
new_words_with_rand = {}
new_words = {}
def getXy(hipon_type, hipers):
    X, y, g = [], [], []
    for i, ((O, typeO), Es) in enumerate(zip(hipon_type, hipers)):
        if typeO == 'Concept':
            try:
                vecO = model.wv[O]
                new_words.setdefault(i, [])
                new_words_with_rand.setdefault(i, [])
                for pos, E in enumerate(Es):
                    size_part = len(Es) / 3.0
                    try:
                        vecE = model.wv[E]
                        X.append(diff(vecO, vecE))
                        
                        part = 3 - int(pos / size_part)
                        y.append(part)
                        
                        new_words[i].append(E)
                        new_words_with_rand[i].append(E)
                    except:
                        for rand_i in range(10):
                            randWord = random.sample(words_model, k=1)[0]
                            vecRand = model.wv[randWord]
                            X.append(diff(vecO, vecRand))
                            y.append(0)


                            new_words_with_rand[i].append(randWord)
                        
                    g.append(i)
            except:
                pass
            
    return X, y, g

In [342]:
words_model = model.wv.vocab.keys()
def create_DF():
    X, y, g = getXy(hipon_type, hipers)
    
    X_train, X_test, y_train,  y_test, g_train, g_test = train_test_split(X, y, g, 0.66)
    
    return X_train, X_test, y_train,  y_test, g_train, g_test

In [343]:
random.seed(37)
def train_test_split(X, y, g, ratio=0.66):
    g_train_set = random.sample(set(g), int(ratio * len(set(g))))
    
    X_train, X_test, y_train,  y_test, g_train, g_test = [], [], [], [], [], []
    for x_i, y_i, g_i in zip(X, y, g):
        if g_i in g_train_set:
            X_train.append(x_i)
            y_train.append(y_i)
            g_train.append(g_i)
        else:
            X_test.append(x_i)
            y_test.append(y_i)
            g_test.append(g_i)

    return X_train, X_test, y_train,  y_test, g_train, g_test

In [344]:
def split_group(groups):
    lens = []
    
    len_g = 1
    g_prev = groups[0]
    for g in groups[1:]:
        if g == g_prev:
            len_g += 1
        else:
            lens.append(len_g)
            len_g = 1
        g_prev = g
    lens.append(len_g)
    return lens

In [345]:
X_train, X_test, y_train,  y_test, g_train, g_test = create_DF()

In [346]:
print len(X_test), len(X_train)

1723 3481


In [347]:
model_LGBM = LGBMRanker(learning_rate=0.1, n_estimators=100, subsample=0.8, max_depth=5, num_leaves = 64, verbose=1)
model_LGBM = model_LGBM.fit(X_train, y_train, group=split_group(g_train))

In [322]:
def compare_predicts(real, predict):
    PF = 0
    NT = 0
    PT = 0
    NF = 0

    dif = []
    for r, p in zip(real, predict):
        if real == 0:
            if predict == 0:
                NT += 1
            else:
                PF += 1
        else:
            if predict == 0:
                NF += 1
            else:
                PT += 1
            dif.append(np.abs(r-p))
    print float(NT) / len(NT + NF)
    print dif.mean()

In [348]:
def write_predict_gold(model, X_test, g_test):
    predicts = model.predict(X_test)
    with open('gold_Google', 'w') as goldF, \
         open('test_Google', 'w') as testF:
            one_line = [predicts[0]]
            g_last = g_test[0]
            for g, p in zip(g_test[1:], predicts[1:]):
                if g == g_last:
                    one_line.append(p)
                else:
                    words = zip(*sorted(zip(one_line, new_words_with_rand[g_last]), key=itemgetter(0), reverse=True))[1]
                    testF.write('\t'.join(words) + '\n')
                
                    goldF.write('\t'.join(new_words[g_last]) + '\n')
                    
                g_last = g

            words = zip(*sorted(zip(one_line, new_words_with_rand[g_last]), key=itemgetter(0), reverse=True))[1]
            testF.write('\t'.join(words) + '\n')
            
            goldF.write('\t'.join(new_words[g_last]) + '\n')

In [349]:
write_predict_gold(model_LGBM, X_test, g_test)
!python SemEval18-Task9/task9-scorer.py ./gold_Google ./test_Google

MRR: 0.365590717924
MAP: 0.313769351636
P@1: 0.28
P@3: 0.305185185185
P@5: 0.313925925926
P@15: 0.321268903936


In [340]:
write_predict_gold(model_LGBM, X_test, g_test)
!python SemEval18-Task9/task9-scorer.py ./gold_Google ./test_Google

MRR: 0.711407407407
MAP: 0.677629479985
P@1: 0.622222222222
P@3: 0.653333333333
P@5: 0.674074074074
P@15: 0.696178599179


#### Fit собствнной модели

In [17]:
def getNsents(ffile, N=1000000):
    sents = []
    flag_EOF = False
    for i in xrange(N):
        sent = ffile.readline()
        if len(sent) == 0:
            flag_EOF = True
            break
            
        words = prepare(sent)
        if len(words) != 0:
            sents.append(words)
        
    return sents, flag_EOF

In [18]:
def fit_model(algorithm, window, size, min_count, alpha):
    N = 1000000
    with open('./UMBC_tokenized.txt') as file_text:
        model = gensim.models.word2vec.Word2Vec(iter=5, sg=algorithm, window=window, size=size, min_count=min_count, alpha=alpha)

        i = 0
        while(True):
            last_time = time()
            sents, flag_EOF = getNsents(file_text, N)

            if i == 0:
                model.build_vocab(sents)
            else:
                model.build_vocab(sents, update=True)
                model.train(sents, total_examples=self.corpus_count, epochs=self.iter)

            i += N
#             logging.info("Step: %d" %i)
            model.save('modelsW2V/my_model_' + str(i / N))

            print "Step: %d, Time: %d" %(i, (time() - last_time) / 60)
            if flag_EOF:
                break
                
                        
            
                
    return model

In [ ]:
algorithm = 1 # Skip-gramm
window = 7
size = 300
min_count = 1
alpha = 0.025

my_model = fit_model(algorithm, window, size, min_count, alpha)

Step: 1000000, Time: 0


In [ ]:
model = gensim.models.Word2Vec.load("./modelsW2V/my_model_18")

00:02:58   loading Word2Vec object from ./modelsW2V/my_model_18
